In [1]:
import sqlite3
import pandas as pd
connection = sqlite3.connect("sql_final.db")
cursor = connection.cursor()

In [2]:
milk = pd.read_csv("milk_production.csv")
milk.head()

,Year,Period,Geo_Level,State_ANSI,Commodity_ID,Domain,Value
0,2023,APR,STATE,4.0,5,TOTAL,"428,000,000"
1,2023,APR,STATE,6.0,5,TOTAL,"3,543,000,000"
2,2023,APR,STATE,8.0,5,TOTAL,"444,000,000"
3,2023,APR,STATE,12.0,5,TOTAL,"166,000,000"
4,2023,APR,STATE,13.0,5,TOTAL,"180,000,000"


In [3]:
milk['Value'] = milk['Value'].str.replace(',', '')
milk.head()

,Year,Period,Geo_Level,State_ANSI,Commodity_ID,Domain,Value
0,2023,APR,STATE,4.0,5,TOTAL,428000000
1,2023,APR,STATE,6.0,5,TOTAL,3543000000
2,2023,APR,STATE,8.0,5,TOTAL,444000000
3,2023,APR,STATE,12.0,5,TOTAL,166000000
4,2023,APR,STATE,13.0,5,TOTAL,180000000


In [4]:
milk['Value'] =pd.to_numeric(milk['Value'])
milk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37638 entries, 0 to 37637
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Year          37638 non-null  int64  
 1   Period        37638 non-null  object 
 2   Geo_Level     37638 non-null  object 
 3   State_ANSI    37634 non-null  float64
 4   Commodity_ID  37638 non-null  int64  
 5   Domain        37638 non-null  object 
 6   Value         37638 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 2.0+ MB


In [5]:
milk.to_sql('milk', con=connection, if_exists='replace', index=False)

37638

In [6]:
coffee = pd.read_csv("coffee_production.csv")
coffee['Value'] = coffee['Value'].str.replace(',', '')
coffee['Value'] = pd.to_numeric(coffee['Value'])
coffee.to_sql('coffee', con=connection, if_exists='replace', index=False)

honey = pd.read_csv("honey_production.csv")
honey['Value'] = honey['Value'].str.replace(',', '')
honey['Value'] = pd.to_numeric(honey['Value'])
honey.to_sql('honey', con=connection, if_exists='replace', index=False)

state = pd.read_csv("state_lookup.csv")
#state.head()
state.to_sql('state', con=connection, if_exists='replace', index=False)

yogurt = pd.read_csv("yogurt_production.csv")
#yogurt.head()
#yogurt['Value'] = yogurt['Value'].str.replace(',', '')
#yogurt.head()
#yogurt.iloc[25]
yogurt['Value'] = yogurt['Value'].str.replace(',', '').str.replace(' (D)', '')
yogurt['Value'] = pd.to_numeric(yogurt['Value'], errors='coerce')
yogurt.to_sql('yogurt', con=connection, if_exists='replace', index=False)

cheese = pd.read_csv("cheese_production.csv")
cheese['Value'] = cheese['Value'].str.replace(',', '').str.replace(' (D)', '')
cheese['Value'] = pd.to_numeric(cheese['Value'], errors='coerce')
cheese.to_sql('cheese', con=connection, if_exists='replace', index=False)

7488

In [7]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[('my_table',),
 ('milk',),
 ('coffee',),
 ('honey',),
 ('state',),
 ('yogurt',),
 ('cheese',)]

### What is the total milk production in year 2023?

In [8]:
cursor.execute("SELECT sum(value) FROM milk WHERE year = 2023").fetchall()

[(91812000000,)]

### What is the avergae honey production in year 2022?

In [9]:
cursor.execute("select avg(value) from honey where year = 2022").fetchall()

[(3133275.0,)]

### What is the ID for IOWA state?

In [10]:

cursor.execute("select * from state where state = 'IOWA'").fetchall()

[('IOWA', 19)]

### Does state with ID 35 produce both honey and milk in 2022?

In [11]:
cursor.execute("select * from honey h inner join milk m on h.state_ANSI=m.state_ANSI where h.year = 2022 and h.state_ANSI=35").fetchall()

[]

### What is the total yogurt production in year 2022 for states also producing cheese in 2022?

In [12]:
cursor.execute("select sum(value) from yogurt where year=2022 and State_ANSI in (select State_ANSI from cheese where year =2022) ").fetchall()

[(1171095000.0,)]

### How many states have total cheese production more than 100 million on April 2023?

In [13]:
cursor.execute("select * from cheese where year = 2023 and period = 'APR' group by State_ANSI having sum(value)>100000000").fetchall()

[(2023, 'APR', 'STATE', None, 6, 'TOTAL', 215206000.0),
 (2023, 'APR', 'STATE', 6.0, 6, 'TOTAL', 208807000.0),
 (2023, 'APR', 'STATE', 55.0, 6, 'TOTAL', 289699000.0)]

### What is the total cheese production for each states on April 2023?

In [14]:
cursor.execute("SELECT sum(c.value), s.state FROM state s left join cheese c on s.State_ANSI=c.State_ANSI where year = 2023 and period = 'APR' group by s.state").fetchall()

[(208807000.0, 'CALIFORNIA'),
 (86452000.0, 'IDAHO'),
 (5068000.0, 'ILLINOIS'),
 (31512000.0, 'IOWA'),
 (69728000.0, 'MINNESOTA'),
 (4889000.0, 'NEW JERSEY'),
 (79038000.0, 'NEW MEXICO'),
 (66256000.0, 'NEW YORK'),
 (20510000.0, 'OHIO'),
 (39420000.0, 'PENNSYLVANIA'),
 (43253000.0, 'SOUTH DAKOTA'),
 (11279000.0, 'VERMONT'),
 (289699000.0, 'WISCONSIN')]

### What is the average coffee production on the year where honey production is larger than 1 million?

In [ ]:
cursor.execute("SELECT AVG(value) FROM COFFEE where year in (select year from honey group by year having sum(value)>1000000)").fetchall()

[(6426666.666666667,)]

In [ ]:
connection.close()